# Import Modules

In [1]:
import os
import sys
import pickle

import pandas as pd

# #############################################################################
sys.path.insert(0, os.path.join(os.environ["PROJ_irox"], "data"))
from proj_data_irox import (
    bulk_dft_data_path, unique_ids_path,
    static_irox_structures_path)

# #############################################################################
from IPython.display import display

In [2]:
with open(bulk_dft_data_path, "rb") as fle:
    df_bulk_dft = pickle.load(fle)

with open(static_irox_structures_path, "rb") as fle:
    df_static = pickle.load(fle)
    
df_ids = pd.read_csv(unique_ids_path)

In [3]:
def method(row_i):
    atoms = row_i["atoms"]
    num_atoms = atoms.get_number_of_atoms()
    return(num_atoms)

df_static["num_atoms"] = df_static.apply(method, axis=1)

In [4]:
row_i = df_static.iloc[0]

atoms = row_i["atoms"]
num_atoms = atoms.get_number_of_atoms()

In [5]:
ab3_ids = df_ids[df_ids["stoich"] == "AB3"]["unique_ids"].tolist()

In [6]:
print("df_static.shape:", df_static.shape)

print("df_bulk_dft.shape:", df_bulk_dft.shape)

df_static.shape: (967, 7)
df_bulk_dft.shape: (587, 9)


# IrO3 Structures

In [7]:
df_static_ab3 = df_static[
    (df_static["stoich"] == "AB3") & \
    (df_static["source"] == "chris") & \
    [True for i in range(len(df_static))]
    ]

In [8]:
df_bulk_dft_ab3 = df_bulk_dft[
    (df_bulk_dft["stoich"] == "AB3") & \
    (df_bulk_dft["source"] == "raul") & \
    [True for i in range(len(df_bulk_dft))]
    ]

# df_bulk_dft_ab3

In [9]:
df_bulk_dft_ab3.index

ab3_ids_not_finished = [i for i in df_static_ab3.index if i not in df_bulk_dft_ab3.index]

df_i = df_static_ab3.loc[ab3_ids_not_finished]

print("Structures with greater than 100 atoms")
df_tmp = df_i[df_i["num_atoms"] >= 100]

display(df_tmp)

Structures with greater than 100 atoms


,id_old,atoms,stoich,path,source,static_id,num_atoms
id_unique,,,,,,,
9k7pvd7izl,158,"(Atom('O', [16.791656436, 1.4750354304000002, ...",AB3,/home/raulf2012/Dropbox/01_norskov/PROJECT_DAT...,chris,liwigegu_28,160
v57txw6dbw,32,"(Atom('O', [4.547374670000002, 10.250076881935...",AB3,/home/raulf2012/Dropbox/01_norskov/PROJECT_DAT...,chris,dupureke_04,144


In [10]:
df_i = df_i.drop(df_tmp.index)

# df_i

In [11]:
# #############################################################################
path_i = os.path.join(
    os.environ["PROJ_DATA"],
    "04_IrOx_surfaces_OER/ml_bulk_irox_dft/iro3",
    "df_dict.pickle")
with open(path_i, "rb") as fle:
    df_dict = pickle.load(fle)
df_iro3_sherlock = df_dict["df"]
df_new_jobs_sherlock = df_dict["df_new_jobs"]
df_iro3_sherlock["cluster"] = "sherlock"

# #############################################################################
# Parsing Sherlock IrO3 DFT Data
path_i = os.path.join(
    os.environ["PROJ_DATA"],
    "04_IrOx_surfaces_OER/ml_bulk_irox_dft/iro3",
    "df_dict_sher.pickle")
with open(path_i, "rb") as fle:
    df_dict = pickle.load(fle)
df_iro3_nersc = df_dict["df"]
df_new_jobs_nersc = df_dict["df_new_jobs"]
df_iro3_nersc["cluster"] = "nersc"

# #############################################################################
df_iro3 = pd.concat([
    df_iro3_nersc,
    df_iro3_sherlock 
    ], axis=0, sort=True)

In [12]:
ids_running_on_nersc = [
    46,
    70,
    83,
    144,
    152,
    173,
    206,
    220,
    228,
    ]

ids_running_on_sher = [
    2,
    22,
    51,
    101,
    250,
    ]

ids_running = ids_running_on_nersc + ids_running_on_sher

len(ids_running)

14

In [13]:
df_i = df_i[~df_i["id_old"].isin(ids_running)]

In [68]:
# df_iro3

# for id_i in df_i["id_old"].tolist():
#     df_tmp = df_iro3[df_iro3["id"] == str(id_i).zfill(3)]

#     print(id_i)
#     display(df_tmp.sort_values("revision"))
#     print(40 * "-")
    
    
completed_ids = df_iro3[
    (df_iro3["completed"] == True) & \
    (df_iro3["isif"] == 2)
    ]["id"].tolist()

not_done_ids = df_iro3[~df_iro3["id"].isin(completed_ids)]["id"].unique().tolist()
for ind_i, id_i in enumerate(not_done_ids):
    df_tmp = df_iro3[df_iro3["id"] == str(id_i).zfill(3)]

    print(ind_i, " | ", id_i)
    display(df_tmp.sort_values("revision"))
    print(40 * "-")


0  |  002


,atoms,cluster,completed,error,error_type,id,isif,job_state,num_steps,path,pre_path,revision,submitted,timed_out
97,"(Atom('Ir', [5.00209, 8.67369, 20.15544], inde...",nersc,None,None,None,002,7.0,None,5,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,1,True,True
99,"(Atom('Ir', [4.11906, 7.14249, 16.59733], inde...",nersc,None,None,None,002,7.0,None,9,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,2,True,True
100,"(Atom('Ir', [3.46895, 6.0152, 13.97774], index...",nersc,True,None,None,002,7.0,SUCCEEDED,9,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,3,True,None
101,"(Atom('Ir', [7.0354, 0.54525, 15.06096], index...",nersc,None,None,None,002,3.0,None,126,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,4,True,True
102,"(Atom('Ir', [12.27377, 0.91087, 14.47342], ind...",nersc,None,None,None,002,3.0,None,124,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,5,True,True
103,"(Atom('Ir', [14.84529, 0.92668, 13.17181], ind...",nersc,None,None,None,002,3.0,None,112,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,6,True,True
104,"(Atom('Ir', [15.09369, 0.95001, 12.75794], ind...",nersc,None,None,None,002,3.0,None,139,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,7,True,True
105,"(Atom('Ir', [15.0997, 0.95382, 12.81683], inde...",nersc,True,None,None,002,3.0,SUCCEEDED,150,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,8,True,None
106,"(Atom('Ir', [15.10858, 0.95376, 12.80607], ind...",nersc,True,None,None,002,3.0,SUCCEEDED,27,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,9,True,None
96,"(Atom('Ir', [15.11055, 0.95384, 12.80649], ind...",nersc,None,None,None,002,3.0,None,348,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,10,True,True


----------------------------------------
1  |  236


,atoms,cluster,completed,error,error_type,id,isif,job_state,num_steps,path,pre_path,revision,submitted,timed_out
124,"(Atom('Ir', [3.96467, 2.28895, 1.31588], index...",nersc,True,None,None,236,7.0,SUCCEEDED,4,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,1,True,None
125,"(Atom('Ir', [3.46141, 2.004, 0.70418], index=0...",nersc,None,True,None,236,3.0,None,16,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,2,True,None
126,"(Atom('Ir', [4.01759, 2.26081, 1.6282], index=...",nersc,True,None,None,236,3.0,SUCCEEDED,150,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,3,True,None
127,"(Atom('Ir', [4.0015, 2.60603, 1.12955], index=...",nersc,True,None,None,236,3.0,SUCCEEDED,150,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,4,True,None
128,None,nersc,None,None,None,236,NaN,None,0,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,5,True,None


----------------------------------------
2  |  250


...

----------------------------------------
11  |  173


,atoms,cluster,completed,error,error_type,id,isif,job_state,num_steps,path,pre_path,revision,submitted,timed_out
1113,"(Atom('Ir', [-3e-05, 3.01827, 0.58255], index=...",sherlock,None,True,None,173,7.0,None,6,/global/cscratch1/sd/flores12/IrOx_Project_tem...,/global/cscratch1/sd/flores12/IrOx_Project_tem...,1,True,None
1108,"(Atom('Ir', [-2e-05, 2.68576, 0.51837], index=...",sherlock,True,None,None,173,7.0,SUCCEEDED,8,/global/cscratch1/sd/flores12/IrOx_Project_tem...,/global/cscratch1/sd/flores12/IrOx_Project_tem...,2,True,None
1117,"(Atom('Ir', [-3e-05, 3.04057, 0.18697], index=...",sherlock,None,None,None,173,3.0,None,12,/global/cscratch1/sd/flores12/IrOx_Project_tem...,/global/cscratch1/sd/flores12/IrOx_Project_tem...,3,True,True
1107,"(Atom('Ir', [-0.07056, 4.0091, -0.24967], inde...",sherlock,None,None,None,173,3.0,None,51,/global/cscratch1/sd/flores12/IrOx_Project_tem...,/global/cscratch1/sd/flores12/IrOx_Project_tem...,4,True,True
1114,"(Atom('Ir', [-0.04243, 4.48051, 7.47634], inde...",sherlock,None,None,None,173,3.0,None,26,/global/cscratch1/sd/flores12/IrOx_Project_tem...,/global/cscratch1/sd/flores12/IrOx_Project_tem...,5,True,True
1109,"(Atom('Ir', [0.15201, 4.72318, 7.51671], index...",sherlock,None,None,None,173,3.0,None,31,/global/cscratch1/sd/flores12/IrOx_Project_tem...,/global/cscratch1/sd/flores12/IrOx_Project_tem...,6,True,True
1116,"(Atom('Ir', [-0.04166, 5.15807, 7.69361], inde...",sherlock,None,None,None,173,3.0,None,74,/global/cscratch1/sd/flores12/IrOx_Project_tem...,/global/cscratch1/sd/flores12/IrOx_Project_tem...,7,True,True
1110,"(Atom('Ir', [-0.38614, 5.16605, 7.82093], inde...",sherlock,None,None,None,173,3.0,None,84,/global/cscratch1/sd/flores12/IrOx_Project_tem...,/global/cscratch1/sd/flores12/IrOx_Project_tem...,8,True,True
1112,"(Atom('Ir', [-0.30784, 5.07856, 7.8604], index...",sherlock,None,None,None,173,3.0,None,126,/global/cscratch1/sd/flores12/IrOx_Project_tem...,/global/cscratch1/sd/flores12/IrOx_Project_tem...,9,True,True
1118,"(Atom('Ir', [-0.33099, 5.25512, 0.00034], inde...",sherlock,None,None,None,173,3.0,None,96,/global/cscratch1/sd/flores12/IrOx_Project_tem...,/global/cscratch1/sd/flores12/IrOx_Project_tem...,10,True,True


----------------------------------------
12  |  144


,atoms,cluster,completed,error,error_type,id,isif,job_state,num_steps,path,pre_path,revision,submitted,timed_out
1190,"(Atom('O', [4e-05, 2.18889, 11.96014], index=0...",sherlock,None,None,None,144,7.0,None,9,/global/cscratch1/sd/flores12/IrOx_Project_tem...,/global/cscratch1/sd/flores12/IrOx_Project_tem...,1,True,True
1188,"(Atom('O', [3e-05, 1.62531, 8.88072], index=0)...",sherlock,None,True,None,144,7.0,None,7,/global/cscratch1/sd/flores12/IrOx_Project_tem...,/global/cscratch1/sd/flores12/IrOx_Project_tem...,2,True,None
1193,"(Atom('O', [3e-05, 1.48133, 8.09405], index=0)...",sherlock,True,None,None,144,7.0,SUCCEEDED,8,/global/cscratch1/sd/flores12/IrOx_Project_tem...,/global/cscratch1/sd/flores12/IrOx_Project_tem...,3,True,None
1187,"(Atom('O', [0.17076, 1.50139, 8.15508], index=...",sherlock,None,None,None,144,3.0,None,145,/global/cscratch1/sd/flores12/IrOx_Project_tem...,/global/cscratch1/sd/flores12/IrOx_Project_tem...,4,True,True
1191,"(Atom('O', [-0.16243, 1.68725, 7.96835], index...",sherlock,None,None,None,144,3.0,None,43,/global/cscratch1/sd/flores12/IrOx_Project_tem...,/global/cscratch1/sd/flores12/IrOx_Project_tem...,5,True,True
1189,"(Atom('O', [-0.55702, 1.82973, 7.72497], index...",sherlock,None,None,None,144,3.0,None,120,/global/cscratch1/sd/flores12/IrOx_Project_tem...,/global/cscratch1/sd/flores12/IrOx_Project_tem...,6,True,True
1192,"(Atom('O', [-0.65215, 1.83558, 7.45763], index...",sherlock,None,None,None,144,3.0,None,120,/global/cscratch1/sd/flores12/IrOx_Project_tem...,/global/cscratch1/sd/flores12/IrOx_Project_tem...,7,True,True


----------------------------------------


['002',
 '236',
 '250',
 '099',
 '070',
 '108',
 '228',
 '220',
 '148',
 '046',
 '256',
 '173',
 '144']

In [56]:
# type(df_iro3.loc[22].iloc[0]["completed"])

# df_iro3["completed"].isnull()